In [1]:
%pylab
%matplotlib inline
import matplotlib.gridspec as gridspec
import wfg

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
## define helper functions
def notDominated(X):
    """
    Return a Boolean vector with as many rows as X indicating whether each
    row of X is not dominated by the other elements of X; ie, whether each
    row is Pareto optimal for X.
    """
    y = zeros(X.shape[0], dtype='bool')
    for n in range(X.shape[0]):
        # Find the number of times on each row where X[i,j] >= X[n,j];
        # y[n] is not dominated if this is bigger than for every row
        y[n] = min(sum(X >= X[n,:], 1)) > 0
    return y

## Method
    - Do RME attainment sample
    - Project onto simplex
    - Project back into P

#### WFG2 problem

In [3]:
# define problem
M = 3                                   # Number of objective
kfactor = 2
lfactor = 2

k = kfactor*(M-1)
l = lfactor*2

func = wfg.WFG2

x_limits = np.array([[0]*(k+l),[i*2 for i in range(1, k+l+1)]])
x_limits

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  4,  6,  8, 10, 12, 14, 16]])

In [4]:
n_draw = 10000
# draw samples
x_draw = np.zeros((n_draw, k+l))
y_draw = np.zeros((n_draw, M))
for n in range(n_draw):
    xi = wfg.random_soln(k, l, func.__name__, seed=0)
    x_draw[n,:] = xi
    y_draw[n,:] = func(xi, k, M)

In [46]:
# define shape function
def wfg_shape_map(x):
    if x.ndim==1:
        x = x.reshape(1,-1)
    return np.array([[f1(xi), f2(xi), f3(xi)] for xi in x])# draw samples

def f1(x):
    x = x.reshape(-1)
    return 2*np.prod([1-cos((xi*np.pi)/2) for xi in x])

def f2(x):
    x = x.reshape(-1)
    return 4*(1-cos((pi/2)*x[0]))*(1-sin((pi/2)*x[1]))
            
def f3(x):
    x = x.reshape(-1)
    return 6*(1-x[0]*cos(5*x[0]*pi)**2)


In [47]:
x_shape = np.random.uniform(0, 1, size=[int(n_draw), 2])
y_shape = wfg_shape_map(x_shape)
assert y_shape.shape[0] == x_shape.shape[0]
assert y_shape.shape[1] == M
x_shape.shape

(10000, 2)

In [48]:
## pareto split
pi_draw = notDominated(y_draw)
di_draw = np.invert(pi_draw)
pi_shape = notDominated(y_shape)
di_shape = np.invert(pi_shape)

In [49]:
fig = plt.figure(figsize=[15,8])
spec = gridspec.GridSpec(ncols=4, nrows=2, figure=fig)
ax0 = fig.add_subplot(spec[0, 0], projection='3d')
ax1 = fig.add_subplot(spec[0, 1], projection='3d', sharex=ax0, sharey=ax0, sharez=ax0)
ax2 = fig.add_subplot(spec[1, 0], projection='3d', sharex=ax0, sharey=ax0, sharez=ax0)
ax3 = fig.add_subplot(spec[1, 1], projection='3d', sharex=ax0, sharey=ax0, sharez=ax0)
ax4 = fig.add_subplot(spec[:, 2:], projection='3d', sharex=ax0, sharey=ax0, sharez=ax0)
# 
ax0.scatter(*y_draw.T, c="C0", s=5)
ax1.scatter(*y_shape.T, c="C1", s=5)
ax2.scatter(*y_draw[pi_draw].T, c="C0", s=5)
ax3.scatter(*y_shape[pi_shape].T, c="C1", s=5)
ax4.scatter(*y_draw[pi_draw].T, c="C0", s=5, alpha=0.1)
ax4.scatter(*y_shape[pi_shape].T, c="C1", s=5, alpha=0.1)

ax0.set_title('drawn')
ax1.set_title('shape')
ax4.set_title('comparison')

Text(0.5, 0.92, 'comparison')

In [63]:
H = 20
sj = np.array([i/H for i in range(H+1)])
print(sj.shape)

(21,)


In [64]:
s = []
for s1 in sj:
    for s2 in sj:
        for s3 in sj:
            if np.round_(s1+s2+s3, decimals=9) == 1.0:
                s.append([s1, s2, s3])
s = np.array(s)

In [65]:
fig = plt.figure(figsize=[16,8])
ax0 = fig.add_subplot(1,2,1, projection='3d')
ax1 = fig.add_subplot(1,2,2, projection='3d')

ax0.scatter(*s.T)
ax1.scatter(*wfg_shape_map(s).T)
ax0.view_init(30, 45)

In [181]:
from scipy.optimize import fsolve

def get_x2(x, s1, s2):
    return ((1-sin((pi/2)*x))/(1-cos((pi/2)*x)))-(s2/s1)

def get_x1(x, x2, s2, s3):
    return (1-sin((pi/2)*x))/(1-cos((pi/2)*x))-(s3/s2)*(1-sin((pi/2)*x2))

def get_m(x,x2, s2, s3):
    return ((1-(x*np.cos(5*pi*x)**2))/(1-cos((pi/2)*x)))-(s3/s2)*((1-sin((pi/2)*x2)))

In [182]:
xM = np.zeros((len(s), 2))
for i, si in enumerate(s+0.01):
    x2 = fsolve(get_x2, x0=0.5, args=(si[0], si[1]))[0]
#     x2 = fsolve(g2, x0=1e-5, args=(si[1], si[2]))
    x1 = fsolve(get_m, x0=0.15, args=(x2, si[1], si[2]))[0]
    xM[i, :] = [x1, x2]

In [183]:
pp = np.array([wfg_shape_map(xi) for xi in xM]).squeeze(1)

In [184]:
ppi = notDominated(pp)

In [186]:
%matplotlib qt
fig = plt.figure(figsize=[10, 10])
ax = fig.gca(projection='3d')
ax.scatter(*pp[ppi].T)
# ax.scatter(*pp.T, c='C1')
ax.view_init(30, 45)

In [127]:
t = np.linspace(0, 1, 1000)


In [131]:
plot(t, get_x2(t, s[0][0], s[0][1]))
# plot(t, get_m(t, s[0][1], s[0][2]))

<ipython-input-73-f35867ed4790>:4: RuntimeWarning: divide by zero encountered in true_divide
  return ((1-sin((pi/2)*x))/(1-cos((pi/2)*x)))-(s2/s1)
<ipython-input-73-f35867ed4790>:4: RuntimeWarning: invalid value encountered in double_scalars
  return ((1-sin((pi/2)*x))/(1-cos((pi/2)*x)))-(s2/s1)


In [135]:
plt.plot(t, [get_x2(ti, s[100][0], s[100][1]) for ti in t])

<ipython-input-73-f35867ed4790>:4: RuntimeWarning: divide by zero encountered in double_scalars
  return ((1-sin((pi/2)*x))/(1-cos((pi/2)*x)))-(s2/s1)
